# Prepare Input Data
We are going to train an MPNN to predict the B3LYP-level atomization energy.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from molgym.mpnn.data import convert_nx_to_dict, make_type_lookup_tables, make_tfrecord
from molgym.utils.conversions import convert_smiles_to_nx
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
import json

RDKit WARNING: [10:38:14] Enabling RDKit 2019.09.3 jupyter extensions


## Get the Data
It is stored on a [GitHub page](https://github.com/globus-labs/g4mp2-atomization-energy) from a previous project

In [2]:
data = pd.read_json('https://github.com/globus-labs/g4mp2-atomization-energy/raw/master/data/output/g4mp2_data.json.gz', 
                   lines=True)
print(f'Downloaded {len(data)} training entries')

Downloaded 130258 training entries


Convert the SMILES to a networkx object

In [3]:
%%time
data['nx'] = data['smiles_0'].apply(lambda x: convert_smiles_to_nx(x, add_hs=True))

CPU times: user 51.3 s, sys: 858 ms, total: 52.1 s
Wall time: 52.1 s


## Save the Data as TF Records
We'll make both a training, validation and test set

In [4]:
test_set = data.query('in_holdout')
print(f'Set aside {len(test_set)} training entries')

Set aside 13026 training entries


In [5]:
train_set, val_set = train_test_split(data.query('not in_holdout'), test_size=0.1, random_state=1)
print(f'Split off {len(train_set)} training and {len(val_set)} validation entries')

Split off 105508 training and 11724 validation entries


Get all of the types observed in the dataset

In [6]:
%%time
atom_types, bond_types = make_type_lookup_tables(data['nx'])

CPU times: user 3.92 s, sys: 3.57 ms, total: 3.93 s
Wall time: 3.92 s


In [7]:
with open('atom_types.json', 'w') as fp:
    json.dump(atom_types, fp)
with open('bond_types.json', 'w') as fp:
    json.dump(bond_types, fp)

Save the data in TF format

In [8]:
for name, dataset in zip(['train', 'valid', 'test'], [train_set, val_set, test_set]):
    dataset = dataset.sample(frac=1.)  # Shuffle contents
    with tf.io.TFRecordWriter(f'{name}_data.proto') as writer:
        for _, entry in tqdm(dataset.iterrows(), desc=name):
            record = convert_nx_to_dict(entry['nx'], atom_types, bond_types)
            for output in ['u0_atom', 'g4mp2_atom']:
                record[output] = entry[output]
            writer.write(make_tfrecord(record))

train: 105508it [00:48, 2180.26it/s]
valid: 11724it [00:05, 2198.52it/s]
test: 13026it [00:05, 2234.92it/s]
